This notebook reproduces a [visualization by the Wall Street Journal](http://graphics.wsj.com/infectious-diseases-and-vaccines/#b02g20t20w15) about the incidence of measles over time, which the brilliant [Brian Granger](https://github.com/ellisonbg) adapted into a [computational example for the Altair library](http://nbviewer.jupyter.org/github/ellisonbg/altair/blob/master/altair/notebooks/12-Measles.ipynb).

[Original source](http://holoviews.org/gallery/demos/bokeh/measles_example.html) (from [`holoviews.org`](http://holoviews.org))

## 1. Import required packages/libraries

In [ ]:
# Required import statements
import numpy as np
import pandas as pd
import holoviews as hv

In [ ]:
# Print version information
print(f'Numpy {np.__version__}')
print(f'Pandas {pd.__version__}')
print(f'Holoviews {hv.__version__}')
# Display Holoviews & Bokeh logos below
hv.extension('bokeh')

## 2. Declaring data

In [ ]:
# Load data from CSV file
CSV_FILE = 'measles_incidence.csv'
data = pd.read_csv(CSV_FILE, skiprows=2, na_values='-')

# Add up data by year over all states
yearly_data = data.drop('WEEK', axis=1).groupby('YEAR').sum().reset_index()

# "Melt" wide table into a tall table
measles = pd.melt(yearly_data, id_vars=['YEAR'], var_name='State', value_name='Incidence')

## 3. Constructing individual plots components

In [ ]:
# Construct heatmap from measles table
heatmap = hv.HeatMap(measles, label='Measles Incidence')

# Use heatmap data to compute annual average & standard deviation across all states
aggregated = hv.Dataset(heatmap).aggregate('YEAR', np.mean, np.std)

# Construct a line plot 
line = hv.Curve(aggregated)
# Construct error bars
bars = hv.ErrorBars(aggregated)

# Construct vertical line annotation
vline = hv.VLine(1963)
# Construct text annotation
text = hv.Text(1964, 800, 'Vaccine introduction', halign='left')

## 4. Putting plot pieces together

In [ ]:
# Construct an overlay showing all plot elements together
overlay = (line * bars * vline * text + heatmap).cols(1)

## 5. Cleaning up the final plot

In [ ]:
# Use .opts method & hv.opts to tune plot elements
overlay.opts(
    hv.opts.VLine(line_color='black'),
    hv.opts.Overlay(width=900, height=200, show_title=False, xrotation=90),
    hv.opts.HeatMap(width=900, height=500, xaxis=None, tools=['hover'],
                invert_yaxis=True, labelled=[], toolbar='above',
                colorbar=True, logz=True, clim=(1, np.nan))
)